## Leetcode Practice Problems

### Design Log Aggregation System
* overview
  + Design the LogAggregator class:
    + LogAggregator(int machines, int services) Initializes the object with machines and services representing the number of machines and services in the datacenter, respectively.
    + void pushLog(int logId, int machineId, int serviceId, String message) Adds a log with id logId notifying that the machine machineId sent a string message while executing the service serviceId.
    + List\<Integer\> getLogsFromMachine(int machineId) Returns a list of ids of all logs added by machine machineId.
    + List\<Integer\> getLogsOfService(int serviceId) Returns a list of ids of all logs added while running service serviceId on any machine.
    + List\<String\> search(int serviceId, String searchString) Returns a list of messages of all logs added while running service serviceId where the message of the log contains searchString as a substring.  
  + Note that:
    + The entries in each list should be in the order they were added, i.e., the older logs should precede the newer logs.
    + A machine can run multiple services more than once. Similarly, a service can be run on multiple machines.
    + All logId may not be ordered.
    + A substring is a contiguous sequence of characters within a string.   
* algorithm
  + First, we need to have an ordered data structure for both machine list and service list
    + getLogsFromMachine and getLogsOfService both return an ordered list of all the log ids
  + search function returns a list of message in logs that contains the searchString as a substring
  + based on the above two requirements, we can set two lists, one for each machine id, the second for each service id to store the log ids added to each of the list when pushLog function is executed
  + to store the log messages, we use a dictionary from log id to log message. When search function is executed, we first fetch a list of log id, and then check for each log id, if its message contains the searchString
  
* solution
  + in the given solution from Leetcode, instead of using lists to store the mapping from service and machine ids to log ids, the solutoin used defaultdict, which is basically the same as the lists we used here since the element index is used as the key to the dictionary. Notice that we have a fixed number of machines and services, and therefore, each service and machine id are guaranteed to be within the index ranges of the lists
  + In addition, in search function, accssing to each service item by list index is also O(1), the same as default dict.

In [62]:
# our solution using lists and dictionary
from typing import List
class LogAggregator:

    def __init__(self, machines: int, services: int):
        self.machine_list = [[] for _ in range(machines)]
        self.service_list = [[] for _ in range(services)]
        self.logs = {}
        

    def pushLog(self, logId: int, machineId: int, serviceId: int, message: str) -> None:
        self.machine_list[machineId].append(logId)
        self.service_list[serviceId].append(logId)
        self.logs[logId] = message        

    def getLogsFromMachine(self, machineId: int) -> List[int]:
        return self.machine_list[machineId]        

    def getLogsOfService(self, serviceId: int) -> List[int]:
        return self.service_list[serviceId]
        

    def search(self, serviceId: int, searchString: str) -> List[str]:
        rs = []
        for log_id in self.service_list[serviceId]:
            if searchString in self.logs[log_id]:
                rs.append(self.logs[log_id])
            
        return rs    
            
        


# Your LogAggregator object will be instantiated and called as such:
# obj = LogAggregator(machines, services)
# obj.pushLog(logId,machineId,serviceId,message)
# param_2 = obj.getLogsFromMachine(machineId)
# param_3 = obj.getLogsOfService(serviceId)
# param_4 = obj.search(serviceId,searchString)

In [63]:
# official solution give by leetcode using defaultdict
class LogAggregator:

    def __init__(self, machines, services):
        self.machines = machines
        self.services = services
        self.logs = defaultdict(str)
        self.logsForMachine = defaultdict(list)
        self.logsForService = defaultdict(list)

    def pushLog(self, logId: int ,machineId: int, serviceId: int, message: str) -> None:
        self.logs[logId] = message
        self.logsForMachine[machineId].append(logId)
        self.logsForService[serviceId].append(logId)

    def getLogsFromMachine(self, machineId: int) -> List[int]:
        return self.logsForMachine[machineId]

    def getLogsOfService(self, serviceId: int) -> List[int]:
        return self.logsForService[serviceId]

    def search(self, service: int, searchString: str) -> List[str]:
        filteredLogs = []
        for logId in self.logsForService[service]:
            if searchString in self.logs[logId]:
                filteredLogs.append(self.logs[logId])
        return filteredLogs

### Design a Rate Limiting System
* Overview
  + A Rate Limiting System can allow a maximum of n requests every t seconds, using an implementation similar to the sliding window algorithm.
  + Given two positive integers n and t, and a non-decreasing stream of integers representing the timestamps of requests, implement a data structure that can check if each request is allowed or not.
  + Implement the RateLimiter class:
    + RateLimiter(int n, int t) Initializes the RateLimiter object with an empty stream and two integers n and t.
    + boolean shouldAllow(int timestamp) Returns true if the current request with timestamp timestamp is allowed, otherwise returns false. 
  + Note that while checking if the current request should be allowed, only the timestamps of requests previously allowed are considered.
* algorithm
  + we need a data structure that can accomodate at most n slots within a fixed time window of t
  + given a time stamp, we need to check if within the time window t, do we have enough time slots to accomodate the current request
  + we use a deque data structure, each time when a request comes, we first popleft all the records beyond the current time stamp - t, and check if we have enough time slot (the nubmer of the remaining items should be less than n), if so, we insert the time slot to the queue and return True, otherwise, we just returns False

In [64]:
# our solution using deque
class RateLimiter:

    def __init__(self, n: int, t: int):
        self.queue = deque()
        self.capacity = n
        self.interval = t        

    def shouldAllow(self, timestamp: int) -> bool:
        # we only care about the requests having timestamp >= timestamp -self.interval+1
        while self.queue and self.queue[0] < timestamp - self.interval + 1:
            self.queue.popleft()
            
        # if we still have space with all requests having timestamps >= timestamp-self.interval +1
        # we add this timestamp to the queue and return True. Otherwise, return Fasle and ignore the request
        if not self.queue or len(self.queue) < self.capacity:
            self.queue.append(timestamp)
            return True
        
        return False
            
# Your RateLimiter object will be instantiated and called as such:
# obj = RateLimiter(n, t)
# param_1 = obj.shouldAllow(timestamp)     

In [65]:
# official solution from leetcode
class RateLimiter:

    def __init__(self, N: int, T: int):
        self.N = N
        self.T = T
        self.allowedRequests = deque()

    # Returns true if there were less than N requests that we allowed in the last T seconds.
    def shouldAllow(self, epochTimeInSeconds: int) -> bool:
        if not self.allowedRequests:
            self.allowedRequests.appendleft(epochTimeInSeconds)
            return True
        while (self.allowedRequests and epochTimeInSeconds - self.allowedRequests[-1] >= self.T):
            self.allowedRequests.pop()
        if len(self.allowedRequests) >= self.N:
            return False
        self.allowedRequests.appendleft(epochTimeInSeconds)
        return True

### Design a Load Distributor
* Overview
  + Design a simple load distributor for a data center that can do the following:
    + Add and remove machines from the cluster.
    + Add applications to run on a machine.
    + Stop applications that are running on a machine.
    + Return a list of the applications running on a machine.
  + Implement the DCLoadBalancer class:
    + DCLoadBalancer() Initializes the object.
    + void addMachine(int machineId, int capacity) Registers a machine with the given machineId and maximum capacity.
    + void removeMachine(int machineId) Removes the machine with the given machineId. All applications running on this machine are automatically reallocated to other machines in the same order as they were added to this machine. The applications should be reallocated in the same manner as addApplication.
    + int addApplication(int appId, int loadUse) Allocates an application with the given appId and loadUse to the machine with the largest remaining capacity that can handle the additional request. If there is a tie, the machine with the lowest ID is used. Returns the machine ID that the application is allocated to. If no machine can handle the request, return -1.
    + void stopApplication(int appId) Stops and removes the application with the given appId from the machine it is running on, freeing up the machine's capacity by its corresponding loadUse. If the application does not exist, nothing happens.
    + List\<Integer\> getApplications(int machineId) Returns a list of application IDs running on a machine with the given machineId in the order in which they were added. If there are more than 10 applications, return only the first 10 IDs.
    
* Example

Input                        
["DCLoadBalancer", "addMachine", "addMachine", "addMachine", "addMachine", "addApplication", "addApplication", "addApplication", "addApplication", "getApplications", "addMachine", "addApplication", "stopApplication", "addApplication", "getApplications", "removeMachine", "getApplications"]
[[], [1, 1], [2, 10], [3, 10], [4, 15], [1, 3], [2, 11], [3, 6], [4, 5], [2], [5, 10], [5, 5], [3], [6, 5], [4], [4], [2]]
Output
[null, null, null, null, null, 4, 4, 2, 3, [3], null, 5, null, 2, [1, 2], null, [6, 1]]

Explanation                                         
DCLoadBalancer dCLoadBalancer = new DCLoadBalancer();                               
dCLoadBalancer.addMachine(1, 1); // Capacity Left: [1]                              
dCLoadBalancer.addMachine(2, 10); // Capacity Left: [1,10]                             
dCLoadBalancer.addMachine(3, 10); // Capacity Left: [1,10,10]                                
dCLoadBalancer.addMachine(4, 15); // Capacity Left: [1,10,10,15]                                 
dCLoadBalancer.addApplication(1, 3); // return 4, Capacity Left: [1,10,10,12]                  
                                     // Machine 4 had the largest capacity left at 15.               
dCLoadBalancer.addApplication(2, 11); // return 4, Capacity Left: [1,10,10,1]                  
                                      // Machine 4 had the largest capacity left at 12.             
dCLoadBalancer.addApplication(3, 6); // return 2, Capacity Left: [1,4,10,1]               
                                     // Machine 2 and 3 had the same largest capacity but machine 2 has the lower ID.
dCLoadBalancer.addApplication(4, 5); // return 3, Capacity Left: [1,4,5,1]                     
                                     // Machine 3 had the largest capacity at 10.                            
dCLoadBalancer.getApplications(2); // return [3], Machine 2 only has application 3.                
dCLoadBalancer.addMachine(5, 10); // Capacity Left: [1,4,5,1,10]                           
dCLoadBalancer.addApplication(5, 5); // return 5, Capacity Left: [1,4,5,1,5]
                                     // Machine 5 had the largest capacity at 10.            
dCLoadBalancer.stopApplication(3); // Capacity Left: [1,10,5,1,5],                          
                                   // Application 3 was running on machine 2.                   
dCLoadBalancer.addApplication(6, 5); // return 2, Capacity Left: [1,5,5,1,5]                   
                                     // Machine 2 had the largest capacity at 10.                   
dCLoadBalancer.getApplications(4); // return [1, 2], Machine 4 has applications 1 and 2.       
dCLoadBalancer.removeMachine(4); // Capacity Left: [1,2,5,-,5]           
                                 // Machine 4 had applications 1 and 2.         
                                 // Application 1 has a load of 3 and is added to machine 2.    
                                 // Application 2 has a load of 11 and cannot be added to any machine so it is removed.        
dCLoadBalancer.getApplications(2); // return [6, 1], Machine 2 has applications 6 and 1.      

* Algorithm                   
  + There are 3 classes, Application, Machine and DCLoadBalancer
  + Appication defines the software/app that can be assigned to a machine
    + it contains id, load and machine_id attributes with the correpsonding get and set methods
  + Machine defines the machine can assign Applications to
    + it has the machine id, remaining capacity and a list/dictionary of Applications it has
    + we define the \_\_lt\_\_ for Heap to fetch the max capacity machine, or the machine with smaller id if capacity are the same
      + note we reverse the other.remain < self.remain while for id, we use self.id < other.id
    + addApp(app) function accepts an app and add it to the app list/dictionary using the following logic
      + if self.remain < app.getLoad(), the machine doesn't have enough capacity, return False
      + add the app to app list/dictionary using app id as the key, or add the app id to the app list
      + deduct the app load from the remain capacity of the machine
      + set the machine id of the app
      + return True
    + removeApp(app) function removes an app from the machine
      + remove the app from app list or dictionary
      + add the app load back to the machine capacity
      + set the machine id of the app to -1
  + DCLoadBalancer
    + it has heap to store the machine available to add apps, and apps and machines maps to store the apps and machines in the LB
    + addMachine(machineId, capacity)
      + this function add machine with machineId and capacity to the LB
      + first construct the machine object from machine id and capacity
      + push the machine object to heap
      + add machine with the id to machines dictionary
    + removeMachine(machineId)
      + this function removes machine with machine id from LB
      + get all app ids from the machine
      + remove machine from LB using its machine id
      + for each app id, call addApplication method (note that the apps are removed from machine, not from LB, these apps are still available in LB's apps list/dictionary)
    + addApplication(appId, loadUse)
      + this function add the app with appId and loadUse to the LB
      + first check heap top to make there is machine with sufficient capacity available for the app. Otherwise, return -1. In addition, use a while loop to heappop all the machines that do not exist in the machines list from the top of the heap
      + heappop the machine that is available for the app, add the app to it, and heappush it back
      + add the app to the LB's apps list/dictionary
      + return the machine id
    + stopApplication(appId)
      + if appId not in apps list of the LB, return
      + retrieve the app from apps list/dictionary using appId
      + remove app from the apps list/dictionary
      + from app, get its associated machine id, and remove the app from that machine
      + heapify the heap to rearrange the machines in the heap
    + getApplications(machineId)
      + return a list of ids of the apps assigned to the machine with machineId
      + from machines dictionary, get the machine from machineId
      + call the machine's getApp() method to get the id list, and returns the first 10 ids
      
    

In [66]:
# version 1
# using list to store applications in machine
from heapq import heappop, heappush, heapify
from typing import List

class Application:
    def __init__(self, app_id: int, load: int, machine_id : int=-1):
        self.app_id = app_id
        self.load = load
        self.machine_id = machine_id
        
    def setMachineId(self, machine_id: int) -> None:
        self.machine_id = machine_id
        
    def getLoad(self) -> int:
        return self.load
    
    def getMachineId(self) -> int:
        return self.machine_id
    
    def getAppId(self) -> int:
        return self.app_id
    
class Machine:
    def __init__(self, m_id: int, capacity: int):
        self.id = m_id
        self.apps = []
        self.remain = capacity
        
    def __lt__(self, other) -> bool:
        if self.remain == other.remain:
            return self.id < other.id
        return other.remain < self.remain
    
    def addApp(self, app: 'Application') -> bool:
        if self.remain < app.getLoad():
            return False
        
        self.apps.append(app.getAppId())
        self.remain -= app.getLoad()
        app.setMachineId(self.id)
        return True
    
    def removeApp(self, app: 'Application' ) -> None:
        self.apps.remove(app.getAppId())
        self.remain += app.getLoad()
        app.setMachineId(-1) 
        
    def getApp(self) -> List['Applicatoin']:
        return self.apps
    
    def getRemain(self) -> int:
        return self.remain
    
    def getId(self) -> int:
        return self.id

class DCLoadBalancer:

    def __init__(self):
        # machine objs list to maintain machine with highest available capacity
        self.heap = []
        
        # dictionary to maintain app ids with application objs
        self.apps = {}
        
        # dictionary to maintain machine ids with machine objs
        self.machines = {}

    def addMachine(self, machineId: int, capacity: int) -> None:
        
        # construct machine obj
        machine = Machine(machineId, capacity)
        
        # push machine obj to heap
        heappush(self.heap, machine)
        
        # add machine obj to dictionary with machine id as the key
        # we can update machine from this dictionary with its key
        self.machines[machineId] = machine
                
    def removeMachine(self, machineId: int) -> None:
        # if the machine id doesn't exist in dictionary, it has been deleted
        if machineId not in self.machines:
            return
        
        # if the heap top contains the machine to be removed, pop the machine from heap
        if self.heap[0].id == machineId:
            heappop(self.heap)
           
        # get all app ids from the machine to be removed to tranfer 
        # delete the entry of this machine from machine dictionary
        apps = self.machines[machineId].getApp()
        del self.machines[machineId]    
        
        # add each app to the max capacity available machines
        for app in apps:
            self.addApplication(app, self.apps[app].getLoad())            

    def addApplication(self, appId: int, loadUse: int) -> int:
                
        # remove all the deleted machines from heap
        while self.heap and self.heap[0].id not in self.machines:
            heappop(self.heap)        
        
        if not self.heap:
            return -1      
        
        # if the app size is bigger than the max available capacity machine, return -1
        if self.heap[0].getRemain() < loadUse:
            return -1
        
        # otherwise, pop the max capacity available machine (successor) from heap, 
        # consturct app obj, add the app to the successor, and push it back to heap
        successor = heappop(self.heap)
        app = Application(appId, loadUse)
        successor.addApp(app)        
        heappush(self.heap, successor)
       
        # register the app to apps dictionary
        self.apps[appId] = app
            
        return successor.getId()   
        

    def stopApplication(self, appId: int) -> None:
        # if the app has been deleted, return
        if appId not in self.apps:
            return
        
        # get the app obj to be deleted from dictionary, and delete its entry
        app = self.apps[appId]
        del self.apps[appId]
        
        # remove the app obj from the corresponding machine obj
        # then heapify the machines in self.heap to maintain the max heap
        self.machines[app.getMachineId()].removeApp(app)       
        heapify(self.heap)       
        

    def getApplications(self, machineId: int) -> List[int]:
        
        # return the first 10 app obj's id for a specific machineId
        return self.machines[machineId].getApp()[:10]
        


# Your DCLoadBalancer object will be instantiated and called as such:
# obj = DCLoadBalancer()
# obj.addMachine(machineId,capacity)
# obj.removeMachine(machineId)
# param_3 = obj.addApplication(appId,loadUse)
# obj.stopApplication(appId)
# param_5 = obj.getApplications(machineId)

In [67]:
# version 2
# using dictionary to store applications in machine
from heapq import heappop, heappush, heapify

class Application:
    def __init__(self, app_id: int, load: int, machine_id : int=-1):
        self.app_id = app_id
        self.load = load
        self.machine_id = machine_id
        
    def setMachineId(self, machine_id: int) -> None:
        self.machine_id = machine_id
        
    def getLoad(self) -> int:
        return self.load
    
    def getMachineId(self) -> int:
        return self.machine_id
    
    def getAppId(self) -> int:
        return self.app_id
    
class Machine:
    def __init__(self, m_id: int, capacity: int):
        self.id = m_id
        self.apps = {}
        self.remain = capacity
        
    def __lt__(self, other) -> bool:
        if self.remain == other.remain:
            return self.id < other.id
        return other.remain < self.remain
    
    def addApp(self, app: 'Application') -> bool:
        if self.remain < app.getLoad():
            return False
        
        self.apps[app.getAppId()] = app
        self.remain -= app.getLoad()
        app.setMachineId(self.id)
        return True
    
    def removeApp(self, app: 'Application' ) -> None:
        del self.apps[app.getAppId()]
        self.remain += app.getLoad()
        app.setMachineId(-1) 
        
    def getApp(self) -> List['Applicatoin']:
        return list(self.apps.keys())
    
    def getRemain(self) -> int:
        return self.remain
    
    def getId(self) -> int:
        return self.id

class DCLoadBalancer:

    def __init__(self):
        # machine objs list to maintain machine with highest available capacity
        self.heap = []
        
        # dictionary to maintain app ids with application objs
        self.apps = {}
        
        # dictionary to maintain machine ids with machine objs
        self.machines = {}

    def addMachine(self, machineId: int, capacity: int) -> None:
        
        # construct machine obj
        machine = Machine(machineId, capacity)
        
        # push machine obj to heap
        heappush(self.heap, machine)
        
        # add machine obj to dictionary with machine id as the key
        # we can update machine from this dictionary with its key
        self.machines[machineId] = machine
                
    def removeMachine(self, machineId: int) -> None:
        # if the machine id doesn't exist in dictionary, it has been deleted
        if machineId not in self.machines:
            return
        
        # if the heap top contains the machine to be removed, pop the machine from heap
        if self.heap[0].id == machineId:
            heappop(self.heap)
           
        # get all app ids from the machine to be removed to tranfer 
        # delete the entry of this machine from machine dictionary
        app_ids = self.machines[machineId].getApp()
        del self.machines[machineId]    
        
        # add each app to the max capacity available machines
        for app_id in app_ids:
            self.addApplication(app_id, self.apps[app_id].getLoad())            

    def addApplication(self, appId: int, loadUse: int) -> int:
                
        # remove all the deleted machines from heap top 
        # this ensures that the application is added to an existing machine
        while self.heap and self.heap[0].id not in self.machines:
            heappop(self.heap)        
        
        if not self.heap:
            return -1      
        
        # if the app size is bigger than the max available capacity machine, return -1
        if self.heap[0].getRemain() < loadUse:
            return -1
        
        # otherwise, pop the max capacity available machine (successor) from heap, 
        # consturct app obj, add the app to the successor, and push it back to heap
        successor_machine = heappop(self.heap)
        app = Application(appId, loadUse)
        successor_machine.addApp(app)        
        heappush(self.heap, successor_machine)
       
        # register the app to apps dictionary
        self.apps[appId] = app
            
        return successor_machine.getId()   
        

    def stopApplication(self, appId: int) -> None:
        # if the app has been deleted, return
        if appId not in self.apps:
            return
        
        # get the app obj to be deleted from dictionary, and delete its entry
        app = self.apps[appId]
        del self.apps[appId]
        
        # remove the app obj from the corresponding machine obj
        # then heapify the machines in self.heap to maintain the max heap
        self.machines[app.getMachineId()].removeApp(app)       
        heapify(self.heap)       
        

    def getApplications(self, machineId: int) -> List[int]:
        
        # return the first 10 app obj's id for a specific machineId
        return self.machines[machineId].getApp()[:10]
        


# Your DCLoadBalancer object will be instantiated and called as such:
# obj = DCLoadBalancer()
# obj.addMachine(machineId,capacity)
# obj.removeMachine(machineId)
# param_3 = obj.addApplication(appId,loadUse)
# obj.stopApplication(appId)
# param_5 = obj.getApplications(machineId)

### Design Whatsapp
* Overview
  + Design a system like Whatsapp with the following features:
    + Send a message to a user.
    + Create a group with some initial users.
    + Add more users to a group.
    + Send a message to a group.
    + Get messages for a user.
  + Implement the WhatsApp class:
    + hatsApp() Initializes the object.
    + oid sendMessage(int toUser, String message) Sends a personal message with the text message to the user with id: toUser.
    + nt createGroup(int[] initialUsers) Creates a new group that initially contains users whose ids are in the list initialUsers, and the group id is returned. For each group created, increment the ids sequentially. For the first group to be created id = 1, for the second group id = 2, and so on.
    + oid addUserToGroup(int groupId, int userId) Adds the user with id: userId to the group with id: groupId. This call should be ignored if the user is already in that group, or if the group does not exist.
    + oid sendGroupMessage(int fromUser, int groupId, String message) Sends a message with the text message by the user with id: fromUser to the group with id: groupId. The message should be sent to all members of the group except the sender. Users added afterwards to the group should not receive the message. Also, this call should be ignored if the user is not a part of the group, or if the group does not exist.
    + List\<String\> getMessagesForUser(int userId) Returns all the personal and group messages that were sent to the user with id: userId ordered by the latest ones first.
    
* Algorithm
  + we need the direct access to users from application, and from groups
    + to do this, we create two dictionaries, one for users, another for groups in application
    + we can directly access users, or access users from groups
    + when add a user to a group, we create a user, and then add the user to the group, and add the mapping of the user in users dictionary.
    + since both the group and application contain users dictionary, we can easy check access the user from both app and groups. Both of the dictionaries refer to the same object reference.
  + since the meassges need to be retrieved as list from the most recent to earlier, we use a deque and use its appendleft() function to insert messages to the top of the queue  
* Offical algorithm and version 2 
  + we actually only need to maintain two defaultdict. 
    + One for groups with group id and the corresponding user ids as a set. 
    + one for users with user id and a list to save the messages
    + we use defaultdict so that we don't need to check the edge cases of
      + creating new users and adding messages or adding messages to an existing user.
      + creating a group and then adding users or adding users to an existing group 
      + The defaultdict will automaitcally create the list or set for us
    + we use a set in each group entry to manage the users so that we don't need to consider a user added to a group repeatedly  

In [68]:
from collections import deque
class User:
    def __init__(self, user_id: int):
        self.user_id = user_id
        self.message = deque()             
        
    def get_message(self) -> List[str]:
        return self.message
    
    def add_message(self, msg: str) -> None:
        self.message.appendleft(msg)  
        
    def get_id(self) -> int:
        return self.user_id
        
class Group:
    def __init__(self, group_id: int):
        self.group_id = group_id
        self.users = defaultdict()         
                    
    def add_user(self, user: 'User') -> None:
        user_id = user.get_id()
        if user_id in self.users:
            return        
        self.users[user_id] = user
        
    def send_message(self, message:str, from_user: int):
        # check if the user is in the group
        if from_user not in self.users:
            return
        
        # access each users in the group and add message
        for id, user in self.users.items():
            if id != from_user:
                user.add_message(message)            
            
class WhatsApp:

    def __init__(self):
        self.next_group = 1
        self.groups = {}
        self.users = {}    
        

    def sendMessage(self, toUser: int, message: str) -> None:
        # if the user doesn't exist, create the user and put it in the users dictionary
        if toUser not in self.users:                     
            self.users[toUser] = User(toUser)   
        
        # add messag to the user
        self.users[toUser].add_message(message)        

    def createGroup(self, initialUsers: List[int]) -> int:
        # create the group
        group_id = self.next_group
        group = Group(group_id)        
        
        # retrieve the user from users dictionary
        # if not exist, create the user and put it in the dictionary
        # then add the user to the group
        for user_id in initialUsers:
            if user_id not in self.users:
                self.users[user_id] = User(user_id)
            user = self.users[user_id]
            group.add_user(user)
        
        # put the group in groups dictionary and increment the next_group id
        self.groups[group_id] = group
        self.next_group += 1
        return group_id        

    def addUserToGroup(self, groupId: int, userId: int) -> None:
        if groupId not in self.groups:
            return
        
        # if the user is a new user, create it and put it in users dictionary
        if userId not in self.users:
            user = User(userId)
            self.users[userId] = user
            
        # retrieve the user and add it to the group
        # it is possible that a user may belong to different groups
        user = self.users[userId]    
        self.groups[groupId].add_user(user)        

    def sendGroupMessage(self, fromUser: int, groupId: int, message: str) -> None:        
        if groupId not in self.groups:
            return
        
        # get the group and send message within the group
        group = self.groups[groupId] 
        group.send_message(message, fromUser)         

    def getMessagesForUser(self, userId: int) -> List[str]:
        # access the user from users dictionary and retrieve messages        
        if userId not in self.users:
            return []
       
        return self.users[userId].get_message()
    
# Your WhatsApp object will be instantiated and called as such:
# obj = WhatsApp()
# obj.sendMessage(toUser,message)
# param_2 = obj.createGroup(initialUsers)
# obj.addUserToGroup(groupId,userId)
# obj.sendGroupMessage(fromUser,groupId,message)
# param_5 = obj.getMessagesForUser(userId)

In [69]:

class WhatsApp:

    def __init__(self):
        # A list of all messages sent to user or the user's groups, in the order they were sent.
        self.messagesForUser = defaultdict(list)

        # A set of all the users in a group.
        self.usersInGroup = defaultdict(set)

        # A running counter to find the next group Id
        self.groupCount = 1


    def sendMessage(self, toUser: int, message: str) -> None:
        self.messagesForUser[toUser].append(message)        

    def createGroup(self, initialUsers: List[int]) -> int:
        newGroupId = self.groupCount
        self.groupCount += 1
        self.usersInGroup[newGroupId] = set(initialUsers)
        return newGroupId


    def addUserToGroup(self, groupId: int, userId: int) -> None:
        if(groupId < self.groupCount):
            self.usersInGroup[groupId].add(userId)

    def sendGroupMessage(self, fromUser: int, groupId: int, message: str) -> None:
        members = self.usersInGroup[groupId]
        if fromUser in members:
            for member in members:
                if member != fromUser:
                    self.sendMessage( member, message)

    def getMessagesForUser(self, userId: int) -> List[str]:
        userMessages = list(self.messagesForUser[userId])
        userMessages.reverse()
        return userMessages


In [70]:
# version 2, use defaultdict to only save the relavant information
class WhatsApp:

    def __init__(self):
        # A list of all messages sent to user or the user's groups, in the order they were sent.
        self.messagesForUser = defaultdict(list)

        # A set of all the users in a group.
        self.usersInGroup = defaultdict(set)

        # A running counter to find the next group Id
        self.groupCount = 1


    def sendMessage(self, toUser: int, message: str) -> None:
        self.messagesForUser[toUser].append(message)        

    def createGroup(self, initialUsers: List[int]) -> int:
        newGroupId = self.groupCount
        self.groupCount += 1
        self.usersInGroup[newGroupId] = set(initialUsers)
        return newGroupId


    def addUserToGroup(self, groupId: int, userId: int) -> None:
        if(groupId < self.groupCount):
            self.usersInGroup[groupId].add(userId)

    def sendGroupMessage(self, fromUser: int, groupId: int, message: str) -> None:
        members = self.usersInGroup[groupId]
        if fromUser in members:
            for member in members:
                if member != fromUser:
                    self.sendMessage( member, message)

    def getMessagesForUser(self, userId: int) -> List[str]:
        userMessages = list(self.messagesForUser[userId])
        userMessages.reverse()
        return userMessages


### Design Facebook
* Overview
  + Design a system like Facebook with the following features:
    + A user can write a post.
    + Two users can become friends with each other.
    + Users can see all the posts written by their friends.
  + Implement the Facebook class:
    + Facebook() Initializes the object.
    + void writePost(int userId, String postContent) The user with id userId writes a post with the content postContent.
    + void addFriend(int user1, int user2) user1 and user2 become friends with each other. This call should be ignored if user1 and user2 are already friends.
    + List\<String\> showPosts(int userId) Returns all the posts made by the friends of the user with id userId ordered by the latest ones first, including ones made before they became friends. Note that the posts made by user userId should not be returned.
* algorithm
    + we can use the sample principles of Desing whatsapp by keep the relavant items in appropriate collections
    + the challenge is to maintain the order of the posts by friends. Here we introduce a dictionary to maintain the post id and post content
      + we create a counter to maintain the post ids in order
      + we then maintain the dictionary of list for each user's post ids
    + we maintain a dictionary of set to for each user's friends
      + whenever two users are friends, we add their ids to each other's friend set
    + when showing the friends' posts, we traverse the user's friend set, and combine the ids of the posts by these friends, sort the ids in reverse order, and then return the list of the post contents from their id and the post dictionary  
    

In [71]:
class Facebook:

    def __init__(self):
        
        # maintain the friend set for each user
        self.friends = defaultdict(set)
        
        # maintain the list of post ids for each user
        self.posts_for_user = defaultdict(list)
        
        # maintain the mapping from id to content for each post
        self.posts = defaultdict(str)
        
        # maintain the post id to ensure each post has a unique id
        self.post_count = 0
        

    def writePost(self, userId: int, postContent: str) -> None:
        # retrieve the current post id
        post_id = self.post_count
        
        # register the post with its id and content
        self.posts[post_id] = postContent
        
        # append the post id to the corresponding user id
        self.posts_for_user[userId].append(post_id)
        
        # increment the post count
        self.post_count += 1       

    def addFriend(self, user1: int, user2: int) -> None:
        
        # update the friend sets
        self.friends[user1].add(user2)
        self.friends[user2].add(user1)        

    def showPosts(self, userId: int) -> List[str]:
        
        # get the list of post ids for all the user's friends
        # sort the ids in reverse order, and retrieve the content
        post_ids = []
        for friend_id in self.friends[userId]:
            post_ids.extend(self.posts_for_user[friend_id])
        post_ids.sort(reverse=True) 
        return [self.posts[id] for id in post_ids]
        


# Your Facebook object will be instantiated and called as such:
# obj = Facebook()
# obj.writePost(userId,postContent)
# obj.addFriend(user1,user2)
# param_3 = obj.showPosts(userId)

### Design a Dating System
* Overview
  + Design a simple dating system like Tinder with the following features:
    + Register a user with their gender, age, preferences, and interests.
    + Find matching users according to their preferred gender, preferred age range, and common interests.
  + Implement the Tinder class:
    + Tinder() Initializes the object.
    + void signup(int userId, int gender, int preferredGender, int age, int minPreferredAge, int maxPreferredAge, List\<String\> interests) Registers a user with the given attributes.
    + List\<Integer\> getMatches(int userId) Returns the top 5 matches for the given user. The returned matches should satisfy the following:
      + The returned user's gender should equal the given user's preferredGender.
      + The returned user's age should be between the given user's minPreferredAge and maxPreferredAge (inclusive).
      + There should be at least 1 common interest between the returned user and the given user.
      + The results should be sorted in decreasing order by the number of common interests. If there is a tie, it should be sorted in increasing order by userId.
      + If there are fewer than 5 matches, return as many as possible.
      + Note that the given user might not necessarily be a match for the returned users.
* Algorithm
  + create a User class to store all the user's attributes, preferred attributes and interests
    + note that we convert the interests from list to set to make it easier to find the overlap of interests of two users by intersetion operation
  + in Tinder class, create a users dictionary to map user ids to user objects
  + in signup function, create the user, and register the user in the users dictionary by user id
  + in getMatches method
    + first check if the users dictionary is empty, or the input userId doesn't exist in the dictionary, if so, return an empty list
    + traverse the users dictionary
      + filter out the users that don't meet the creteria
      + heappush the users meeting the minimum requirements
    + output the ids of top 5 matched users by nsmallest() function of heapq 
  

In [72]:
from typing import List
class User:
    def __init__(self, userId:int, gender: int, preferredGender: int, age: int, minPreferredAge: int, maxPreferredAge: int, interests: List[str]):
        self.userId = userId
        self.gender = gender
        self.preferredGender = preferredGender
        self.age = age
        self.minPreferredAge = minPreferredAge
        self.maxPreferredAge = maxPreferredAge
        self.interests = set(interests)
        
        
class Tinder:

    def __init__(self):
        self.users = {}        

    def signup(self, userId: int, gender: int, preferredGender: int, age: int, minPreferredAge: int, maxPreferredAge: int, interests: List[str]) -> None:
        
        # create the user and register it in the users dictionary
        user = User(userId, gender, preferredGender, age, minPreferredAge, maxPreferredAge, interests)
        self.users[user.userId] = user        

    def getMatches(self, userId: int) -> List[int]:
        
        # if there is no users signed up or the input userId doesn't exists, return []
        if not self.users or userId not in self.users:
            return []
        
        # initialize a min heap for ordering the top 5 recommendations
        heap = []
        
        # find the input user from userId, and extract the preferences
        user = self.users[userId]
        preferredGender, minPreferredAge, maxPreferredAge, interests = user.preferredGender, user.minPreferredAge, user.maxPreferredAge, user.interests
        
        # traverse the users dictionary for other users
        for id, other_user in self.users.items():
            
            # filter out unmatched users
            if id == userId or other_user.age < minPreferredAge or other_user.age > maxPreferredAge or other_user.gender != preferredGender or not interests.intersection(other_user.interests):
                continue
                
            # push the matched users to the min heap ordered by negative of matched interests and id
            heapq.heappush(heap, (-len(interests.intersection(other_user.interests)), id))  
            
        # heappop the ids of top 5 recommendations
        return [ id for (_, id) in heapq.nsmallest(5, heap) ]          


# Your Tinder object will be instantiated and called as such:
# obj = Tinder()
# obj.signup(userId,gender,preferredGender,age,minPreferredAge,maxPreferredAge,interests)
# param_2 = obj.getMatches(userId)

### Design Twitter
* Overview
  + Design a simplified version of Twitter where users can post tweets, follow/unfollow another user, and is able to see the 10 most recent tweets in the user's news feed.
  + mplement the Twitter class:
    + Twitter() Initializes your twitter object.
    + void postTweet(int userId, int tweetId) Composes a new tweet with ID tweetId by the user userId. Each call to this function will be made with a unique tweetId.
    + List\<Integer\> getNewsFeed(int userId) Retrieves the 10 most recent tweet IDs in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user themself. Tweets must be ordered from most recent to least recent.
    + void follow(int followerId, int followeeId) The user with ID followerId started following the user with ID followeeId.
    + void unfollow(int followerId, int followeeId) The user with ID followerId started unfollowing the user with ID followeeId.
* Algorithm 
  + we need to maintain the following three dictionaries
    + tweets_of_user maintains the tweet ticket number for each user. The tweet ticket number is used to track the order of tweets posted
    + follows maintains the followee set for each user
    + tweets_ticket maintains the mapping from tweet ticket number to the tweet id
  + we created the tweet_count to generate and keep the track of tweet ticket number 
  + when a tweet is posted
    + we retrieve the current tweet ticket number, and append the number to the user's tweet list
    + we then establish the mapping between the ticket number and the tweet id
    + increment the tweet count
  + when a user add a followee, we add the followee's id to the user's followee set
  + when a user removes a followee, we remove the followee's id from the user's followee set using discard() method of set, since some unfollow operates when the followeeId is not in the flollower's set list, meaning that the follower never followed followee    

In [73]:
class Twitter:

    def __init__(self):
        # maintain the tweet tickets for each user
        self.tweets_of_user = defaultdict(list)
        
        # maintain followee set for each user
        self.follows = defaultdict(set)
        
        # maintain mapping from tweet ticket to tweet id
        self.tweets_ticket = defaultdict(int)
        
        # maintain tweet ticket order
        self.tweet_count = 0
             

    def postTweet(self, userId: int, tweetId: int) -> None:
        # retrieve the current tweet ticket number
        tweet_ticket = self.tweet_count
        
        # append the ticket number to the user
        self.tweets_of_user[userId].append(tweet_ticket)
        
        # establish the mapping between the ticket number and tweet id
        self.tweets_ticket[tweet_ticket] = tweetId
        
        # increment tweet count
        self.tweet_count += 1 
        

    def getNewsFeed(self, userId: int) -> List[int]:
        # create a tmp variable to collect all tweet tickets
        tweet_tickets = []
        # first updates the tweets of the user
        tweet_tickets += self.tweets_of_user[userId]
        
        # updates the tweets posted by the followees
        for followee_id in self.follows[userId]:
            tweet_tickets.extend(self.tweets_of_user[followee_id])
        
        # sort the tweet tickets to maintain the order of tweets
        tweet_tickets.sort(reverse=True)
        
        # retrieve the tweet ids and returns the top 10
        return [self.tweets_ticket[ticket] for ticket in tweet_tickets[:10] ]       

    def follow(self, followerId: int, followeeId: int) -> None:
        # add followee id to the follower's followee set
        self.follows[followerId].add(followeeId)        

    def unfollow(self, followerId: int, followeeId: int) -> None:
        # use discard to remove the followeeid from followee set
        # in case followee id dosen't exist in the set, discard just ignore the operation
        self.follows[followerId].discard(followeeId)        


# Your Twitter object will be instantiated and called as such:
# obj = Twitter()
# obj.postTweet(userId,tweetId)
# param_2 = obj.getNewsFeed(userId)
# obj.follow(followerId,followeeId)
# obj.unfollow(followerId,followeeId)

### Leetcode 535 Encode and Decode TinyURL
* Overview
  + TinyURL is a URL shortening service where you enter a URL such as https://leetcode.com/problems/design-tinyurl and it returns a short URL such as http://tinyurl.com/4e9iAk. Design a class to encode a URL and decode a tiny URL.
  + There is no restriction on how your encode/decode algorithm should work. You just need to ensure that a URL can be encoded to a tiny URL and the tiny URL can be decoded to the original URL.
  + Implement the Solution class:
    + Solution() Initializes the object of the system.
    + String encode(String longUrl) Returns a tiny URL for the given longUrl.
    + String decode(String shortUrl) Returns the original long URL for the given shortUrl. It is guaranteed that the given shortUrl was encoded by the same object.
* Algorithm
  + we implmented the fixed length random string algorithm
  + we use all the upper and lower case of ascii char and the 10 digits. Therefore, we have 62 chars as the base chars
  + we use a fixed length of 6 for each short url string, which provides 62^6 = 5.6 * 10^10 possible combinations
  + we established two dictionaries, one for long urls that maps the long urls to its short urls. The other for short urls that maps the short urls to the long urls
  + when we encode a long url, we first check the long url dictionary if this long url has been encoded, if so, we return the short url value stored in the dictionay. Otherwise, we generate the random string with a fixed length of 6, check if the generaed string has been used in the short url dictionary, if so, re-generate the string, otherwise, update the short and long url dictionaries, and return the generated short url
  + when decode a short url, we directly return the value from short url dictionary. We use the defaultdict to take care of the case where the shorr url input doesn't exist

In [74]:
from string import ascii_letters, digits
from random import randint
class Codec:
    
    # define class level const
    LETTERS = ascii_letters + digits
    SHORT_URL_HEAD = "http://tinyurl.com/"
    
    def __init__(self):
        # define dictionaries for long urls and short urls
        self.long_urls = {}
        # returns empty string when a short url is not in the keys
        self.short_urls = defaultdict(str)
    
    # this function generates random strings with length defined by size
    def generate_chars(self, size: int) -> str:
        rs = ""
        for _ in range(size):
            index = randint(0, len(Codec.LETTERS)-1)
            rs += Codec.LETTERS[index]
        return rs

    def encode(self, longUrl: str) -> str:
        """Encodes a URL to a shortened URL.
        """
        # if the longUrl already exists, return the short url
        if longUrl in self.long_urls:
            return Codec.SHORT_URL_HEAD + self.long_urls[longUrl]
            
        # repeated generate the string of 6 chars to make sure the short url generated is unique
        while (True):
            tmp = self.generate_chars(6)
            if not self.short_urls or tmp not in self.short_urls:
                self.short_urls[tmp] = longUrl
                self.long_urls[longUrl] = tmp
                return Codec.SHORT_URL_HEAD + tmp
        

    # return the long url stored in short url dictionary
    def decode(self, shortUrl: str) -> str:
        """Decodes a shortened URL to its original URL.
        """
        return self.short_urls[shortUrl.replace(Codec.SHORT_URL_HEAD, "")]

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.decode(codec.encode(url))

### Design Uber
* Overview
  + Implement a Cab Booking Application (like Uber) which facilitates:
    + Addition of new cabs to the system.
    + Updating the trips of various customers.
    + Finding the nearest cabs to a location.
  + Design the Uber class:
    + Uber() Initializes the Uber object with 0 cabs and 0 running trips.
    + void addCab(int cabX, int cabY) Adds a cab located at point (cabX, cabY) to the system. Note that multiple cabs can be at the same location.
    + int[] startTrip(int customerID, int customerX, int customerY) Returns an integer array [nearX, nearY] where nearX and nearY represent the X-coordinate and Y-coordinate (respectively) of the closest available cab to customer customerID, present at (customerX, customerY). In case there are multiple such cabs, it returns the location of the cab with the smallest X-coordinate, and if there are still multiple choices, it chooses the cab with the smallest Y-coordinate. In case there are no available cabs, returns \[-1, -1\]. The cab is then assigned to the customer, who starts their trip.
    + void endTrip(int customerID, int customerX, int customerY) The customer customerID ends their trip at (customerX, customerY). In case a cab was assigned to them by the system, re-adds it back to the system at (customerX, customerY), otherwise ignores the call.
    + List\<List\<Integer\>\> getNearestCabs(int k, int x, int y) Returns a list of locations of the k closest available cabs to (x, y), sorted in non-decreasing order by X-coordinate and subsequently by Y-coordinate. In case there are multiple choices, it chooses the cab with the smaller X-coordinate, and if there are still multiple choices, it chooses the one with the smaller Y-coordinate. In case there are less than k cabs available, it returns the locations of all of them.
    + Note: The distance between two points on the X-Y plane is the Euclidean distance (i.e., √(x1 - x2)2 + (y1 - y2)2).
* Algorithm
  + matintain dictionaries and cab_count
    + maintain a dict for cabas between cab id and the position since multiple cabs can be in the same position
    + maintain a dict to keep the cab id for customer id
    + maintain a cab count to keep track of the cab id
  + define a method to retrieve the k cloestes neibhbours to a specific location with x and y coordinates
    + heappush all the avaialable cabs to the heap, and returns the k smallest cab ids
    + define a class with its dunder lt function used by heapq to order the cab instances
  + startTrip
    + get the closest cab from the coordination of customer_x and customer_y
    + register that cab id to the corresponding customer id
    + remove the cab id from the available cab dictionary
  + endTrip
    + retrieve the cab id from the cutomer id
    + regiser the cab id back to available cabs dictionary with the new cutomer_x and customer_y
    + remove the customer id from the cab_for_customer dictionary
  + knearest cab
    + call internal \_getKnearest() to get cab ids, and returns the sorted list by x and y coordinates 
    
    
  

In [75]:
class CabDist:
    def __init__(self, dist: int, x: int, y: int, id:int):
        self.dist = dist
        self.x = x
        self.y = y
        self.id = id
        
    def __lt__(self, other: 'CabDist') -> bool:
        if self.dist != other.dist:
            return self.dist < other.dist
        elif self.x != other.x:
            return self.x < other.x
        elif self.y != other.y:
            return self.y < other.y
        else:
            return self.id < other.id
        
class Uber:

    def __init__(self):
        self.cab_for_customer = defaultdict(int)
        self.available_cabs = {}
        self.cab_count = 0
        

    def addCab(self, cabX: int, cabY: int) -> None:
        cab_id = self.cab_count
        self.available_cabs[cab_id] = [cabX, cabY]
        self.cab_count += 1

    def _getNearestCabs(self, k: int, x: int, y:int) -> List[int]:
        if not self.available_cabs:
            return []
        
        heap = []
        for id, (pos_x, pos_y) in self.available_cabs.items():
            dist = abs(pos_x - x) + abs(pos_y - y)
            heapq.heappush(heap, CabDist(dist, pos_x, pos_y, id))
        
        return [cab.id for cab in heapq.nsmallest(k, heap)]
        
    def startTrip(self, customerID: int, customerX: int, customerY: int) -> List[int]:
        if not self.available_cabs:
            return [-1, -1]
        
        cab_id = self._getNearestCabs(1, customerX, customerY)[0]
        self.cab_for_customer[customerID] = cab_id
        
        rs = self.available_cabs[cab_id]
        del self.available_cabs[cab_id]
        
        return rs       

    def endTrip(self, customerID: int, customerX: int, customerY: int) -> None:
        if customerID not in self.cab_for_customer:
            return
        
        cab_id = self.cab_for_customer[customerID]
        del self.cab_for_customer[customerID]
        
        self.available_cabs[cab_id] = [customerX, customerY]        

    def getNearestCabs(self, k: int, x: int, y: int) -> List[List[int]]:
        ids = self._getNearestCabs(k, x, y)
        return sorted([self.available_cabs[id] for id in ids], key = lambda x:(x[0], x[1]))
        


# Your Uber object will be instantiated and called as such:
# obj = Uber()
# obj.addCab(cabX,cabY)
# param_2 = obj.startTrip(customerID,customerX,customerY)
# obj.endTrip(customerID,customerX,customerY)
# param_4 = obj.getNearestCabs(k,x,y)

### Design Walnut
* Overview
  + Design a banking SMS parsing and analytics application (like Walnut) which reads all the text messages received by users and analyzes the following:
    + The income and expenditure of a user.
    + The average income and expenditure of all users.
    + Text messages will be represented as a string, with words separated by ' '. The texts will be analyzed and will be   considered valid if it satisfies the following criteria:

  + It contains words of exactly one of the following groups:
    + one or more of "credit", "credited", "deposit", or "deposited" to indicate an earning, or
    + one or more of "debit", "debited", "withdraw", "withdrawal", or "withdrawn" to indicate an expenditure.
    + It has exactly one occurrence of amount. It can be denoted as "USD x", "x USD", "USDx", `$ x`, `x $`, or `$x`, where:
      + x denotes the denomination of the amount. It should lie in the range [0, 109]. Please note that it means  "10000000000000000 USD" is an invalid amount.
      + x can have up to 2 decimal places. Please note that this means "$ 1.0005" is not a valid amount as it has more than 2 decimal places.
    

  + Design the Walnut class:

    + Walnut() Initializes the Walnut object with 0 users and 0 text messages.
    + void parseText(int userID, String text) Analyzes the text message represented as text, and updates it for the user userID if it is a valid text.
    + double getTotalUserEarnings(int userID) Returns the total earnings of user userID. In case no     + valid texts have been analyzed for userID, returns 0.
    + double getTotalUserExpenses(int userID) Returns the total expenses of user userID. In case no     + valid texts have been analyzed for userID, returns 0.
    + double getAverageUserEarnings() Returns the average earnings of all users whose texts have been analyzed and are valid (including users with only expenses), or 0 if no texts have been analyzed.
    + double getAverageUserExpenses() Returns the average expenses of all users whose texts have been analyzed and are valid (including users with only earnings), or 0 if no texts have been analyzed.
  + Note that all answers within 10-5 of the actual answer will be considered accepted.
  
* Algorithm
  + use Enum to define two dictionaries for credit and debit
    + self.audit_log is a dictionary containing two dictionaries marked as CREDIT and DEBIT, respectively
  + instance method can access static method from self
  + an easy way to parse string to float is to define a method returns boolean. If the substring can be converted to float, returns True, otherwise, in catch clause (except ValueError, return False). Then in app function, use float to conver it again, after the check function returns True
  + in find_amount function, we check the three cases:
    + USD or dollar sign has exact match with the current word
      + if index of USD or dollar sign < length -1, check text(i+1), and if it can be converted to float, return the converted float
      + if index of USD or dollar sign > 0, check text(i-1), and if it can be converted to float, return the converted float
    + if USD or dollar sign is in the current word
      + replace both USD and dollar sign by empty string, and the replaced string is not empty
      + check if the string can be converted to float, and return the converted float if so
    + return 0 for all other cases  

In [76]:
 from enum import Enum

class Type(Enum):
    CREDIT = 1
    DEBIT = 2


class Walnut:

    debit_identifiers: [str] = ["debit", "debited", "withdraw", "withdrawal", "withdrawn"]
    credit_identifiers: [str] = ["credit", "credited", "deposit", "deposited"]

    def __init__(self):
        # All unique users in the system
        self.user_ids: [int] = []

        # Stores the total amount credited or debited for a user.
        # two dictionaries, one for credit, the other for debit
        self.audit_log = {
            Type.CREDIT: {},
            Type.DEBIT: {}
        }

        # Total amount of credit across all the users
        self.total_credit: int = 0

        # Total amount of debit across all the users
        self.total_debit: int = 0

    def parseText(self, userID: int, text: str) -> None:
        is_it_debit: bool = self.is_debit(text)
        is_it_credit: bool = self.is_credit(text)
        amount: float = self.find_amount(text)

        if is_it_credit and not is_it_debit:
            self.total_credit = self.total_credit + amount
            user_records: {int, int} = self.audit_log.get(Type.CREDIT)
            if userID in user_records:
                amount = amount + user_records[userID]
            user_records[userID] = amount
            if userID not in self.user_ids:
                self.user_ids.append(userID)
        elif is_it_debit and not is_it_credit:
            self.total_debit = self.total_debit + amount
            user_records: {int, int} = self.audit_log.get(Type.DEBIT)
            if userID in user_records:
                amount = amount + user_records[userID]
            user_records[userID] = amount
            if userID not in self.user_ids:
                self.user_ids.append(userID)

    def is_credit(self, text: str) -> bool:
        for identifier in self.credit_identifiers:
            if identifier in text:
                return True
        return False

    def is_debit(self, text: str) -> bool:
        for identifier in self.debit_identifiers:
            if identifier in text:
                return True
        return False
    
    @staticmethod
    def is_numeric(string: str) -> bool:
        try:
            float(string)
            return True
        except ValueError:
            return False

    def getTotalUserEarnings(self, userID: int) -> float:
        user_records: {int, int} = self.audit_log.get(Type.CREDIT)
        amount: int = 0
        if userID in user_records:
            amount = user_records[userID]
        return amount

    def getTotalUserExpenses(self, userID: int) -> float:
        user_records: {int, int} = self.audit_log.get(Type.DEBIT)
        amount: int = 0
        if userID in user_records:
            amount = user_records[userID]
        return amount

    def find_amount(self, text: str) -> float:
        text = text.replace(",", "")
        words: [str] = text.split(" ")
        counter: int = 0
        for n in words:
            if n == "USD" or n == "$":
                # For the cases like USD 40/ $ 40
                if counter < len(words)-1:
                    if self.is_numeric(words[counter + 1]):
                        number: float = float(words[counter + 1])
                        return number
                # For the cases like 40 $/ 40 USD
                if counter > 0:
                    if self.is_numeric(words[counter-1]):
                        number: float = float(words[counter-1])
                        return number
                counter = counter + 1
            # For the cases like $40, 40USD
            elif "USD" in n or "$" in n:
                num: str = n.replace("USD", "")
                num = num.replace("$", "")
                number: float = 0
                if self.is_numeric(num):
                    if len(num) > 0:
                        number = float(num)
                return number
        return 0

    def getAverageUserEarnings(self) -> float:
        if len(self.user_ids) == 0:
            return 0
        return self.total_credit / len(self.user_ids)

    def getAverageUserExpenses(self) -> float:
        if len(self.user_ids) == 0:
            return 0
        return self.total_debit / len(self.user_ids)
        
   

### Design a Todo List
* Overview
  + Design a Todo List Where users can add tasks, mark them as complete, or get a list of pending tasks. Users can also add tags to tasks and can filter the tasks by certain tags.
  + Implement the TodoList class:
    + TodoList() Initializes the object.
    + int addTask(int userId, String taskDescription, int dueDate, List\<String\> tags) Adds a task for the user with the ID userId with a due date equal to dueDate and a list of tags attached to the task. The return value is the ID of the task. This ID starts at 1 and is sequentially increasing. That is, the first task's id should be 1, the second task's id should be 2, and so on.
    + List\<String\> getAllTasks(int userId) Returns a list of all the tasks not marked as complete for the user with ID userId, ordered by the due date. You should return an empty list if the user has no uncompleted tasks.
    + List\<String\> getTasksForTag(int userId, String tag) Returns a list of all the tasks that are not marked as complete for the user with the ID userId and have tag as one of their tags, ordered by their due date. Return an empty list if no such task exists.
    + void completeTask(int userId, int taskId) Marks the task with the ID taskId as completed only if the task exists and the user with the ID userId has this task, and it is uncompleted.
    
* Algorithm
  + set up a task_for_user dictionary to maintain the task list for each user
  + maintain a task_count to keep the track of task id
  + create a Task class since there are many attributs in tasks, and implement \_\_lt\_\_ for sorting
  + use list comprehesion to retrieve the tasks with filter conditions. Note to use task for task in clause
  + sort the task. Note that sort() returns None
  + mark tasks as completed when traversing the task list for users in completeTask function

In [77]:
# define Task class to fill in all attributes and provide __lt__ method for sorting
class Task:
    
    def __init__(self, taskId: int, userId: int, taskDescription: str, dueDate: int, tags: List[str]):
        self.userId = userId
        self.description = taskDescription
        self.dueDate = dueDate
        self.tags = tags
        self.taskId = taskId
        self.incompleted = True
        
    def __lt__(self, other) -> bool:
        return self.dueDate < other.dueDate
        
class TodoList:

    def __init__(self):
        # create a dictionary to maintain the task list for each user
        self.task_for_user = defaultdict(list)
        
        # keep track of task id by task_count
        self.task_count = 1        

    def addTask(self, userId: int, taskDescription: str, dueDate: int, tags: List[str]) -> int:
        # retrieve the current task_count as the id of the new task
        task_id = self.task_count
        
        # create the new task, and append to the user's task list
        task = Task(task_id, userId, taskDescription, dueDate, tags)
        self.task_for_user[userId].append(task)   
        
        # increment the task_count
        self.task_count += 1
        return task_id

    def getAllTasks(self, userId: int) -> List[str]:
        if userId not in self.task_for_user:
            return []
        tasks = [task for task in self.task_for_user[userId] if task.incompleted]
        
        if not tasks:
            return []
        
        tasks.sort()                 
        return [task.description for task in tasks]
        

    def getTasksForTag(self, userId: int, tag: str) -> List[str]:
        if userId not in self.task_for_user:
            return []
        tasks = [task for task in self.task_for_user[userId] if task.incompleted and tag in task.tags]
        
        if not tasks:
            return []
        
        tasks.sort()
        return [task.description for task in tasks]        

    def completeTask(self, userId: int, taskId: int) -> None:
        if userId in self.task_for_user:
            tasks = self.task_for_user[userId]
            if not tasks:
                return 
            for task in tasks:
                if task.taskId == taskId:
                    task.incompleted = False
                    
        


# Your TodoList object will be instantiated and called as such:
# obj = TodoList()
# param_1 = obj.addTask(userId,taskDescription,dueDate,tags)
# param_2 = obj.getAllTasks(userId)
# param_3 = obj.getTasksForTag(userId,tag)
# obj.completeTask(userId,taskId)